In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install langchainhub
!pip install openai
!pip install python-dotenv
!pip install langchain
!pip install langchain_community
!pip install -qU langchain-openai
!pip install faiss-cpu
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv('/content/drive/Othercomputers/내 노트북/code/MarketingAI-Bot/.env')

True

In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import os

loader = CSVLoader(file_path="/content/drive/Othercomputers/내 노트북/code/MarketingAI-Bot/output.csv")
data = loader.load()


GPT_API_KEY = os.environ.get('GPT_API_KEY')
ORG_ID = os.environ.get('ORG_ID')

In [5]:
data[0]

Document(metadata={'source': '/content/drive/Othercomputers/내 노트북/code/MarketingAI-Bot/output.csv', 'row': 0}, page_content="Index: 1\n생성시간: 2024.07.05 08:52:57\n게시글내용: 𝙉𝙚𝙬𝙡𝙮 𝙡𝙖𝙪𝙣𝙘𝙝!어글리 러블리 마스크팩 트래블러스 키트 출시💃어글리 러블리 마스크 팩 3종 모두를파우치에 쏙, 여행 시 휴대가 간편한30ml 미니어처 사이즈로 만나보세요.사랑스러운 패키지 디자인을 적용해소중한 사람들에게, 늘 고마운 나에게선물해보시는 건 어떨까요?*마스크팩 3종 트래블러스 키트는 카카오톡 선물하기에서 만나보실 수 있습니다.\n해쉬태그: ['#어글리러블리', '#uglylovely']\nimg_path: ./images\\0.jpg")

In [6]:
from langchain_openai import OpenAIEmbeddings

# OpenAI의 "text-embedding-ada-002" 모델을 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002",openai_api_key=GPT_API_KEY)

In [7]:
from langchain_community.vectorstores import FAISS

# 저장
vectorstore = FAISS.from_documents(data, embeddings)

# 저장된 DB의 리트리버를 가져와서 검색에 사용한다.
# 서치 타입 mmr은 유사한 정보를 가진 문서는 제외하는 알고리즘을 사용하는것이다.
# 검색인자로 k는 반환할 문서의 수를 지정한다.
# retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5})

# 유사도를 이용한 retiever 생성
retriever = vectorstore.as_retriever(
    # 검색 유형을 "유사도 점수 임계값"으로 설정합니다.
    search_type="similarity_score_threshold",
    # 검색 인자로 점수 임계값을 0.5로 지정합니다.
    search_kwargs={
        "score_threshold": 0.5,
        "k": 7},
)

In [8]:
# FAISS 모듈에서 인덱스를 가져옴
index = vectorstore.index

# 인덱스에 저장된 벡터의 개수를 출력
vector_count = index.ntotal
print(f"백터스토어에 저장된 데이터 개수: {vector_count}")

백터스토어에 저장된 데이터 개수: 76


In [27]:
# 검색된 컨택스트를 하나로 합쳐준다.
def format_docs(docs):
  result= []
  for doc in docs:
    page_content = doc.page_content
    result.append(page_content)
    # print(page_content)
  return "\n\n".join(result)

# 문서 검색 및 포맷팅
def search_and_format(retriever,question):
    docs = retriever.invoke(question)
    context = format_docs(docs)
    return {"context": context, "question": question}

In [28]:
docs = retriever.invoke("유자")

In [29]:
import json
import re

def convert_to_json(string):
    # 문자열에서 각 요소를 추출하는 정규식 패턴
    pattern = r"Index: (\d+)\n생성시간: ([\d.:\s]+)\n게시글내용: ([^\n]+)\n해쉬태그: \[([^\]]+)\]\nimg_path: ([^\n]+)"
    match = re.search(pattern, string)

    if not match:
        raise ValueError("String format is incorrect")

    index = int(match.group(1))
    생성시간 = match.group(2)
    게시글내용 = match.group(3)
    해쉬태그 = match.group(4).replace("'", "").split(", ")
    img_path = match.group(5).replace("\\", "/")

    # JSON 객체 생성
    json_data = {
        "Index": index,
        "생성시간": 생성시간,
        "게시글내용": 게시글내용,
        "해쉬태그": 해쉬태그,
        "img_path": img_path
    }
    return json_data



In [30]:
context_and_question = search_and_format(retriever,"유자처럼 상큼한것이 없을까?")
print("-"*100)
context_and_question

----------------------------------------------------------------------------------------------------


{'context': "Index: 7\n생성시간: 2024.06.11 07:47:31\n게시글내용: 유자청을 닮은 찐~득한 제형이피부를 맑고 환하게 밀착케어 해줍니다.새콤 달콤한 유자 향으로 즐기는 향기 테라피는 덤!🍋\n해쉬태그: ['#어글리러블리', '#uglylovely']\nimg_path: ./images\\6.jpg\n\nIndex: 62\n생성시간: 2023.12.04 07:52:28\n게시글내용: 유기농, 무농약으로 키워 삐뚤빼뚤 개성있는 생김새로 자란 못난이 무화과에 주목!는 상품 가치가 떨어져 버려지는 못난이 무화과에서 추출한 업사이클링 원료를 사용합니다.\n해쉬태그: ['#어글리러블리피그마스크', '#어글리러블리', '#uglylovely']\nimg_path: ./images\\61.jpg\n\nIndex: 9\n생성시간: 2024.06.07 05:31:50\n게시글내용: 그려 나 고흥 유자여!전남 고흥 못난이 유자를 업사이클링한 원료로 만든어글리 러블리 유자 마스크가 당신의 피부를 맑고 환하게\n해쉬태그: ['#어글리러블리', '#uglylovely']\nimg_path: ./images\\8.jpg\n\nIndex: 65\n생성시간: 2023.11.28 09:46:57\n게시글내용: 화학비료와 농약을 사용하지 않아 자연적으로 생긴 삐뚤빼뚤 영광의 상처를 가진 못난이 유자에 주목!는 상품 가치가 떨어져 버려지는 못난이 유자에서 추출한 업사이클링 원료를 사용합니다.네이버 스토어 단독 𝟮𝟵% 𝗢𝗙𝗙👆🏻상단 프로필 링크 클릭!\n해쉬태그: ['#어글리러블리유자마스크', '#어글리러블리', '#uglylovely']\nimg_path: ./images\\64.jpg\n\nIndex: 21\n생성시간: 2024.04.30 08:21:40\n게시글내용: 유자청을 닮아 찐~득한 텍스처를 가진어글리 러블리 유자 마스크 팩지친 피부에 깊은 보습감과 은은한 광채감으로생기 있는 피부를 경험해보세요.\n해쉬태그: ['#어글리러블리', '#uglylovely

In [39]:
# 랭스미스의 허브에 들어가면 미리 정의된 다양한 프롬프트들이 있다.
# hum를 통해 해당 프롬프트 탬플릿을 사용할 수 있다.
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [48]:
prompt.format(context = "sdf", question ="adf")

"Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: adf \nContext: sdf \nAnswer:"

In [32]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableSequence

llm = ChatOpenAI(model="gpt-4-turbo",openai_api_key=GPT_API_KEY)

rag_chain = (
    {"context":retriever | format_docs, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [33]:
rag_chain.invoke("카카오톡 선물하기 입점을 편의성 관점으로 홍보하는 홍보글을 길게 작성해줘")

'카카오톡 선물하기를 통해 어글리 러블리의 아름다움 제품을 손쉽게 만나보세요! 이제 카카오톡 앱에서 몇 번의 클릭만으로 어글리 러블리의 다양한 뷰티 제품들을 확인하고, 입점 기념으로 제공되는 최대 50%의 할인 혜택을 누릴 수 있습니다. 카카오톡 선물하기는 바쁜 일상 속에서도 손쉽게 선물을 고르고, 주변 사랑하는 사람들에게 특별한 선물을 할 수 있는 편리한 방법을 제공합니다.'

In [49]:
query = "유자"
docs = retriever.invoke(query)

llm = ChatOpenAI(model="gpt-4-turbo",openai_api_key=GPT_API_KEY)

result = prompt.format(context=format_docs(docs), question= query)
print(result)
result = llm( result)
print(result)
arr= []
for doc in docs:
  arr.append(convert_to_json(doc.page_content))
response_data = {"section1":arr,"section2":result.content}
print(response_data)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: 유자 
Context: Index: 8
생성시간: 2024.06.10 06:30:42
게시글내용: 유자C의 브라이트닝 케어🤩세린, 아르기닌을 공융혼합한독자 개발 원료*를 함유한 유자 마스크 팩으로 효과적인 각질 케어와 함께 매끈해진 피부결을 경험해보세요.(*특허 제10-2269121호)
해쉬태그: ['#어글리러블리', '#uglylovely']
img_path: ./images\7.jpg

Index: 7
생성시간: 2024.06.11 07:47:31
게시글내용: 유자청을 닮은 찐~득한 제형이피부를 맑고 환하게 밀착케어 해줍니다.새콤 달콤한 유자 향으로 즐기는 향기 테라피는 덤!🍋
해쉬태그: ['#어글리러블리', '#uglylovely']
img_path: ./images\6.jpg

Index: 63
생성시간: 2023.11.30 08:05:33
게시글내용: 유자스팀 생기톤업 비건 온열 보습팩💡뚜껑을 열자마자 상큼 달달 유자향이 물씬,유자청을 떠올리게 하는 찐~득한 텍스처에피부에 전해지는 은은한 온열감까지!어글리 러블리와 함께 하는 모든 순간의 오감을 자극합니다.네이버 스토어 단독 𝟮𝟵% 𝗢𝗙𝗙👆🏻상단 프로필 링크 클릭!
해쉬태그: ['#어글리러블리', '#uglylovely']
img_path: ./images\62.jpg

Index: 9
생성시간: 2024.06.07 05:31:50
게시글내용: 그려 나 고흥 유자여!전남 고흥 못난이 유자를 업사이클링한 원료로 만든어글리 러블리 유자 마스크가 당신의 피부를

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


content='유자는 상큼하고 새콤 달콤한 향을 가지고 있으멀로, 이를 활용한 다양한 스킨케어 제품에서 향기 테라피 효과를 제공합니다. 이러한 제품들은 유자의 추출물을 함유하여 피부를 맑고 환하게 밀착 케어 해주며, 생기 있고 건강한 피부로 가꾸는 데 도움을 줍니다. 또한, 못난이 유자를 업사이클링한 원료를 사용하여 환경 보호에도 기여합니다.' response_metadata={'token_usage': {'completion_tokens': 193, 'prompt_tokens': 1831, 'total_tokens': 2024}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_486730399b', 'finish_reason': 'stop', 'logprobs': None} id='run-0184c233-4090-4cd3-b19f-391a9bd9f3a3-0' usage_metadata={'input_tokens': 1831, 'output_tokens': 193, 'total_tokens': 2024}
content='유자는 상큼하고 새콤 달콤한 향을 가지고 있으멀로, 이를 활용한 다양한 스킨케어 제품에서 향기 테라피 효과를 제공합니다. 이러한 제품들은 유자의 추출물을 함유하여 피부를 맑고 환하게 밀착 케어 해주며, 생기 있고 건강한 피부로 가꾸는 데 도움을 줍니다. 또한, 못난이 유자를 업사이클링한 원료를 사용하여 환경 보호에도 기여합니다.' response_metadata={'token_usage': {'completion_tokens': 193, 'prompt_tokens': 1831, 'total_tokens': 2024}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_486730399b', 'finish_reason': 'stop', 'logprobs': None} id='run-0184c233-4090-

In [54]:

response_data = {"section1":arr,"section2":result.content}
response_data

{'section1': [{'Index': 8,
   '생성시간': '2024.06.10 06:30:42',
   '게시글내용': '유자C의 브라이트닝 케어🤩세린, 아르기닌을 공융혼합한독자 개발 원료*를 함유한 유자 마스크 팩으로 효과적인 각질 케어와 함께 매끈해진 피부결을 경험해보세요.(*특허 제10-2269121호)',
   '해쉬태그': ['#어글리러블리', '#uglylovely'],
   'img_path': './images/7.jpg'},
  {'Index': 7,
   '생성시간': '2024.06.11 07:47:31',
   '게시글내용': '유자청을 닮은 찐~득한 제형이피부를 맑고 환하게 밀착케어 해줍니다.새콤 달콤한 유자 향으로 즐기는 향기 테라피는 덤!🍋',
   '해쉬태그': ['#어글리러블리', '#uglylovely'],
   'img_path': './images/6.jpg'},
  {'Index': 63,
   '생성시간': '2023.11.30 08:05:33',
   '게시글내용': '유자스팀 생기톤업 비건 온열 보습팩💡뚜껑을 열자마자 상큼 달달 유자향이 물씬,유자청을 떠올리게 하는 찐~득한 텍스처에피부에 전해지는 은은한 온열감까지!어글리 러블리와 함께 하는 모든 순간의 오감을 자극합니다.네이버 스토어 단독 𝟮𝟵% 𝗢𝗙𝗙👆🏻상단 프로필 링크 클릭!',
   '해쉬태그': ['#어글리러블리', '#uglylovely'],
   'img_path': './images/62.jpg'},
  {'Index': 9,
   '생성시간': '2024.06.07 05:31:50',
   '게시글내용': '그려 나 고흥 유자여!전남 고흥 못난이 유자를 업사이클링한 원료로 만든어글리 러블리 유자 마스크가 당신의 피부를 맑고 환하게',
   '해쉬태그': ['#어글리러블리', '#uglylovely'],
   'img_path': './images/8.jpg'},
  {'Index': 64,
   '생성시간': '2023.11.29 08:04: